In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.appName('Challenge 3').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/23 18:46:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
path = 'airtrafficdata/airtraffic_all/airtraffic-part/flightmonth=200801'

In [3]:
airtraffic = spark.read.parquet(path)

In [4]:
airtraffic.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- Car

In [20]:
airtraffic.select('IsDepDelayed').distinct().show()

+------------+
|IsDepDelayed|
+------------+
|         YES|
|          NO|
+------------+



In [6]:
# Get all the flights which are departed late but arrived early or on time (IsArrDelayed is NO).

# Output should contain - FlightCRSDepTime, UniqueCarrier, FlightNum, Origin, Dest, DepDelay, ArrDelay

# FlightCRSDepTime need to be computed using Year, Month, DayOfMonth, CRSDepTime

# FlightCRSDepTime should be displayed using yyyy-MM-dd HH:mm format.

# Output should be sorted by FlightCRSDepTime and then by the difference between DepDelay and ArrDelay

# Also get the count of such flights

In [30]:
airtraffic. \
    filter((col('IsDepDelayed') == 'YES') & (col('Cancelled') == '0') & (col('IsArrDelayed') == 'NO')). \
    select(date_format(to_timestamp(concat(col('Year'),
                          lpad(col('Month'),2,'0'),
                          lpad(col('DayofMonth'),2,'0'),
                          lpad(col('CRSDepTime'),4,'0')),'yyyyMMddHHmm'),'yyyy-MM-dd HH:mm'). \
           alias('FlightCRSDepTime'),'UniqueCarrier','FlightNum','Origin','Dest','DepDelay','ArrDelay'). \
    orderBy('FlightCRSDepTime',col('DepDelay') - col('ArrDelay')). \
    show()

+----------------+-------------+---------+------+----+--------+--------+
|FlightCRSDepTime|UniqueCarrier|FlightNum|Origin|Dest|DepDelay|ArrDelay|
+----------------+-------------+---------+------+----+--------+--------+
|2008-01-01 00:55|           DL|      560|   LAX| ATL|      15|      -4|
|2008-01-01 01:00|           AA|     2466|   SFO| DFW|       2|      -7|
|2008-01-01 03:00|           CO|      488|   SJU| EWR|       3|     -12|
|2008-01-01 04:00|           B6|      724|   BQN| MCO|       4|     -23|
|2008-01-01 05:15|           XE|     2140|   SLC| IAH|       1|     -10|
|2008-01-01 05:25|           XE|     2429|   MSP| IAH|       1|      -4|
|2008-01-01 05:30|           XE|     2785|   BTR| IAH|       6|       0|
|2008-01-01 05:40|           OO|     5793|   CIC| SFO|       1|      -4|
|2008-01-01 05:40|           UA|      422|   SEA| DEN|       4|     -10|
|2008-01-01 06:00|           DL|      970|   FLL| ATL|       2|       0|
|2008-01-01 06:00|           AA|     1734|   SAT| D

In [31]:
airtraffic. \
    filter((col('IsDepDelayed') == 'YES') & (col('Cancelled') == '0') & (col('IsArrDelayed') == 'NO')). \
    select(date_format(to_timestamp(concat(col('Year'),
                          lpad(col('Month'),2,'0'),
                          lpad(col('DayofMonth'),2,'0'),
                          lpad(col('CRSDepTime'),4,'0')),'yyyyMMddHHmm'),'yyyy-MM-dd HH:mm'). \
           alias('FlightCRSDepTime'),'UniqueCarrier','FlightNum','Origin','Dest','DepDelay','ArrDelay'). \
    orderBy('FlightCRSDepTime',col('DepDelay')-col('ArrDelay')). \
    count()

54233